# Deploy a un endpoint online

In [ ]:
! pip show azure-ai-ml

## Conectar a workspace

In [1]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

Found the config file in: D:\Shared\MyTrainingRepos\azure-machine-learning\3.Despliegue_Modelos\labs\config.json


Conectado al Workspace: naturgyml4


## Definir y crear un endpoint

Definir el endpoint. 
> Tener en cuenta que el nombre debe ser único: Usaremos la función `dateTime` para ello.

In [2]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for MLflow diabetes model",
    auth_mode="key",
)

Crear el endpoint:

In [3]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-05261452636155.northcentralus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-05261452636155.northcentralus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-05261452636155', 'description': 'Online endpoint for MLflow diabetes model', 'tags': {}, 'properties': {'createdBy': 'Ricardo Ahumada', 'createdAt': '2025-05-26T12:52:57.909593+0000', 'lastModifiedAt': '2025-05-26T12:52:57.909593+0000', 'azureml.onlineendpointid': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/providers/microsoft.machinelearningservices/workspaces/naturgyml4/onlineendpoints/endpoint-05261452636155', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/providers/Microsoft.MachineLearningServices/locations/northcentralus/mfeOperationsStatus/oeidp:e4ba3a3a-3776-4c8a-981f-845d8977c1a2:62064

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> ¡IMPORTANTE!¡Esperar hasta que el endpoint se cree con éxito antes de continuar!
Una notificación verde debería aparecer en el estudio. </p>



## Configurar el deployment gestionado

In [4]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# create a blue deployment
model = Model(
    path="./model",
    type=AssetTypes.MLFLOW_MODEL,
    description="my sample mlflow model",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="STANDARD_DS1_V2",
    instance_count=1,
)

## Crear el deployment

In [5]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Instance type STANDARD_DS1_V2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint endpoint-05261452636155 exists


.......................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpoint-05261452636155', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/providers/Microsoft.MachineLearningServices/locations/northcentralus/mfeOperationsStatus/odidp:e4ba3a3a-3776-4c8a-981f-845d8977c1a2:9d0449cc-0c4a-459f-a872-ef1c0811d52e?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourceGroups/naturgy/providers/Microsoft.MachineLearningServices/workspaces/naturgyml4/onlineEndpoints/endpoint-05261452636155/deployments/blue', 'Resource__source_path': '', 'base_path': 'd:\\Shared\\MyTrainingRepos\\azure-machine-learning\\3.Despliegue_Modelos\\labs\\lab2', 'creation_context': <azure.ai.ml._restclient.v2023_04_01_preview.models._models_p

Asignar el 100% del tráfico.

In [6]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-05261452636155.northcentralus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-05261452636155.northcentralus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-05261452636155', 'description': 'Online endpoint for MLflow diabetes model', 'tags': {}, 'properties': {'createdBy': 'Ricardo Ahumada', 'createdAt': '2025-05-26T12:52:57.909593+0000', 'lastModifiedAt': '2025-05-26T13:01:40.368436+0000', 'azureml.onlineendpointid': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/providers/microsoft.machinelearningservices/workspaces/naturgyml4/onlineendpoints/endpoint-05261452636155', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/providers/Microsoft.MachineLearningServices/locations/northcentralus/mfeOperationsStatus/oeidp:e4ba3a3a-3776-4c8a-981f-845d8977c1a2:29b52

## Probar el deploymnent

In [7]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

Diabetic


## Listar los endpoints

In [8]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

endpoint-05261452636155


## Obtener detalles de un endpoint

In [9]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'blue': 100}
https://endpoint-05261452636155.northcentralus.inference.ml.azure.com/score


## Borrar el endpoint y deployment

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)